## 諸々インポート

In [1]:
import os
import chainer
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer import cuda
from twitter import *

from sklearn.datasets import fetch_mldata
import numpy as np
xp = cuda.cupy

## Twitterの設定

In [2]:
TOKEN = os.environ["DTA_TWITTER_TOKEN"]
TOKEN_SECRET = os.environ["DTA_TWITTER_TOKEN_SECRET"]
CONSUMER_KEY = os.environ["DTA_TWITTER_CONSUMER_KEY"]
CONSUMER_SECRET = os.environ["DTA_TWITTER_CONSUMER_SECRET"]

twi = Twitter(
  auth = OAuth(TOKEN, TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET))

## モデル

In [3]:
class MyModel(Chain):
    def __init__(self):
        super(MyModel,self).__init__(
            l1 = L.Linear(None,100),
            l2 = L.Linear(None,100),
            l3 = L.Linear(None,10))

    def __call__(self,x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        return self.l3(h)

## 自分で作ったExtension

In [4]:
def myTweetExtension(target_epoch, _log_report='LogReport'):
    @training.make_extension()
    def tweet(trainer):
        print("My Extension!!")
        log_report = trainer.get_extension(_log_report)
        if len(log_report.log) > 0:
            plog = log_report.log[len(log_report.log)-1]
            e = plog['epoch']
            te = target_epoch
            ls = round(plog['main/loss'], 6)
            t = round(plog['elapsed_time'],2)
            status = "インテリジェントなスーパーAI「えぽっく:{0}/{1}, ろす:{2}, じかん:{3}[s]」".format(e,te,ls,t)
            res = twi.statuses.update(status=status)
    return tweet

## 学習するとこ

In [5]:
def train():
    model = L.Classifier(MyModel())
    dev = 0
    if dev >= 0:
        chainer.cuda.get_device(dev).use()
        model.to_gpu()

    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    train, test = chainer.datasets.get_mnist()
    train_iter = chainer.iterators.SerialIterator(train, 200)
    test_iter = chainer.iterators.SerialIterator(test, 200,repeat=False, shuffle=False)

    epoch = 100
    updater = training.StandardUpdater(train_iter, optimizer, device=dev)
    trainer = training.Trainer(updater, (epoch, 'epoch'), out="result")

    # 自分で作ったエクステンション
    interval = (50, 'epoch')
    trainer.extend(myTweetExtension(epoch), trigger=interval)
    
    # Evaluator
    trainer.extend(extensions.Evaluator(test_iter, model, device=dev))

    # LogReport
    trainer.extend(extensions.LogReport())

    # PrintReport
    trainer.extend(extensions.PrintReport( entries=['epoch', 'main/loss', 'main/accuracy', 'elapsed_time' ]))

    print("run")
    trainer.run()


In [6]:
train()

run
epoch       main/loss   main/accuracy  elapsed_time
1           0.416937    0.88415        1.65764       
2           0.172333    0.949634       2.60494       
3           0.12331     0.963751       3.55276       
4           0.0962747   0.971334       4.48265       
5           0.0779234   0.976734       5.49525       
6           0.0654635   0.98           6.49417       
7           0.0526592   0.984084       7.45356       
8           0.0453604   0.98625        8.40997       
9           0.0368609   0.989133       9.50249       
10          0.0324923   0.99025        10.5683       
11          0.0282837   0.991349       11.534        
12          0.02389     0.992882       12.4945       
13          0.02041     0.994132       13.4521       
14          0.0164471   0.995083       14.3975       
15          0.0135287   0.996316       15.3526       
16          0.0125601   0.996549       16.3074       
17          0.0145302   0.995232       17.2542       
18          0.0104193   0.